In [ ]:
import os,sys,json
import kfp

In [ ]:
components_url = "/mnt/dkube/pipeline/components/"
dkube_training_op = kfp.components.load_component_from_file(components_url + "training/component.yaml")
dkube_serving_op = kfp.components.load_component_from_file(components_url + "serving/component.yaml")
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(existing_token=token)

In [69]:
@kfp.dsl.pipeline(
    name='set-envvars-pl',
    description='pl with limits and requests '
)
def limits_requests_pipeline(token,limits_cpu,limits_memory,requests_cpu,requests_memory):
    
    a=[{"EPOCHS":"1"},{"LIMITS_CPU":str(limits_cpu)},{"LIMITS_MEM":str(limits_memory)},{"REQUESTS_CPU":str(requests_cpu)},{"REQUESTS_MEM":str(requests_memory)}]
    

    train       = dkube_training_op(token, '{"image":"ocdr/dkube-datascience-tf-cpu:v2.0.0"}',
                                    framework="tensorflow", version="2.0.0",
                                    program="dkube-examples", run_script="python mnist/train.py",
                                    datasets='["mnist"]', outputs='["mnist"]',
                                    input_dataset_mounts='["/mnist"]',
                                    output_mounts='["/model"]',
                                    envs=json.dumps(a))

In [70]:
client.create_run_from_pipeline_func(limits_requests_pipeline, arguments={"token":token,"limits_cpu": "100","limits_memory": "1000Gi", "requests_cpu": "100","requests_memory": "1000Gi"})

RunPipelineResult(run_id=fd591b27-21cb-4ed4-812d-e2d094140aa8)